In [1]:
from dotenv import load_dotenv
load_dotenv()


True

In [5]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model='gemini-1.5-pro')

llm.invoke("대한민국의 수도는?")



AIMessage(content='대한민국의 수도는 서울입니다.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-0aa41ef6-08ea-4432-9c50-4565f97456e5-0', usage_metadata={'input_tokens': 7, 'output_tokens': 10, 'total_tokens': 17, 'input_token_details': {'cache_read': 0}})

In [ ]:
from langchain_core.prompts import PromptTemplate

model = ChatGoogleGenerativeAI(model='gemini-1.5-flash')
prompt = PromptTemplate.from_template(
    "예/아니오 로 질문에 대답하세요. {question}는 과일입니까?"    
)

chain = prompt | llm

chain.invoke("지우개")


AIMessage(content='아니요.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-1efacd2e-3602-4d96-a738-530085580507-0', usage_metadata={'input_tokens': 22, 'output_tokens': 4, 'total_tokens': 26, 'input_token_details': {'cache_read': 0}})

In [15]:
### 책의 정보를 알려주는 인공지능 에이전트를 만들어 보세요

from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field

class BookInfo(BaseModel):
    title: str = Field(description="도서 제목")  # LLM이 description을 반영
    author: str = Field(description="글쓴이")
    publication_year: int = Field(description="출판 연도")
    summary: str = Field(description="내용 요약")

parser = PydanticOutputParser(pydantic_object=BookInfo)
llm = ChatGoogleGenerativeAI(model='gemini-1.5-flash')
prompt = PromptTemplate.from_template(
    """
    사용자가 입력한 도서의 정보를 알려주세요
    도서의 제목, 글쓴이, 출판 연도를 알려주세요
    도서의 간단한 내용을 200자 이내로 요약하세요
    도서 이름: {book_name}
    출력:{format}
    """
)
# partial varible: 사용자가 아닌 개발자가 접근하는 변수
prompt = prompt.partial(format=parser.get_format_instructions())

chain = (
    prompt | llm | parser
)

output = chain.invoke("이기적 유전자")
print(output)

output = chain.invoke("해리포터")
print(output)

print(type(output))


title='이기적 유전자' author='리처드 도킨스' publication_year=1976 summary="이기적 유전자는 유전자의 관점에서 생명체의 행동을 설명하는 책입니다.  진화의 과정을 유전자의 관점에서 해석하며, 생명체는 유전자의 생존 기계일 뿐이라는 주장을 펼칩니다.  자연선택은 개체가 아닌 유전자 수준에서 이루어지며, 유전자는 자신의 복제를 위해 이기적으로 행동한다는 것이 핵심 내용입니다.  이는 개체의 이타적인 행동도 유전자의 이기적인 전략으로 설명될 수 있다는 것을 의미합니다.  종종 오해되는 부분이지만, '이기적'은 유전자의 행동 방식을 설명하는 용어이며, 인간의 도덕성과는 무관합니다."
title='해리포터와 마법사의 돌' author='조앤 K. 롤링' publication_year=1997 summary='고아로 살아가던 해리 포터는 11살 생일에 자신이 마법사이며 호그와트 마법학교에 입학하게 된다는 사실을 알게 된다. 호그와트에서 해리는 친구인 론 위즐리와 헤르미온느 그레인저를 만나고, 마법 세계를 배우며 여러 모험을 겪는다.  특히 어둠의 마법사 볼드모트와의 싸움이 중심 내용이며, 해리는 자신의 부모를 죽인 볼드모트와 맞서 싸우는 과정에서 용기와 우정, 사랑을 배우게 된다.  마법사의 돌을 둘러싼 음모와 해리의 성장 이야기가 흥미진진하게 펼쳐진다.'
<class '__main__.BookInfo'>
